In [3]:
%pip install sounddevice scipy numpy transformers librosa

   ---------------------------------------- 0.0/363.6 kB ? eta -:--:--
   --- ------------------------------------ 30.7/363.6 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------  358.4/363.6 kB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 363.6/363.6 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sounddevice as sd
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

# ตั้งค่าพารามิเตอร์การบันทึก
SAMPLE_RATE = 16000  # ความถี่ของเสียงที่โมเดลรองรับ
DURATION = 5  # ระยะเวลาบันทึกเสียง (วินาที)

# ฟังก์ชันสำหรับบันทึกเสียงจากไมโครโฟน
def record_audio(duration, sample_rate):
    print("เริ่มบันทึกเสียง...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype="float32")
    sd.wait()  # รอให้การบันทึกเสร็จสมบูรณ์
    print("บันทึกเสียงเสร็จสิ้น!")
    return np.squeeze(audio)



In [7]:
# โหลดโมเดลและ processor
model_name = "airesearch/wav2vec2-large-xlsr-53-th"  # โมเดลที่รองรับภาษาไทย
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# บันทึกเสียง
audio = record_audio(DURATION, SAMPLE_RATE)

# แปลงเสียงให้พร้อมใช้งานกับโมเดล
inputs = processor(audio, sampling_rate=SAMPLE_RATE, return_tensors="pt", padding=True)

# ใช้โมเดลถอดรหัสเสียง
with torch.no_grad():
    logits = model(inputs.input_values).logits

# ถอดรหัสข้อความจากโมเดล
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]

# แสดงผลลัพธ์
print("ข้อความที่ถอดรหัสได้:")
print(transcription)

เริ่มบันทึกเสียง...
บันทึกเสียงเสร็จสิ้น!
ข้อความที่ถอดรหัสได้:
ฉัน รู้สึก ไม่ ดี   ฉัน รู้สึก ปวดหัว เล็กน้อย ช่วง นี้


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# โหลด Tokenizer และโมเดลที่ Fine-tune ไว้
tokenizer = AutoTokenizer.from_pretrained("path_to_thai_bert_model")
model = AutoModelForSequenceClassification.from_pretrained("path_to_thai_bert_model")

# ข้อความจากขั้นตอน STT
text = transcription

# แปลงข้อความเป็น Token
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# วิเคราะห์ข้อความ
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)
labels = ["ปกติ", "เครียด", "ซึมเศร้า"]  # หมวดหมู่
print(f"ผลลัพธ์: {labels[predictions]}")
